*Note: Place this notebook in STHN directory.*

# TGM (Satellite to Thermal)

In [ ]:
%pip install PyYAML transformers timm h5py torch torchvision faiss-cpu einops opencv-python

## Image

[my_eval_pix2pix.py](./global_pipeline/my_eval_pix2pix.py)
- [Input Image](./global_pipeline/my_eval_pix2pix.py:69)
- [Output Image](./global_pipeline/my_eval_pix2pix.py:94)


In [ ]:
!py ./global_pipeline/my_eval_pix2pix.py --resume="./maps/models/TGM_nocontrast/best_model.pth" --dataset_name=none --datasets_folder ./maps --G_net unet --GAN_upsample bilinear --GAN_resize 1024 1024

## Folder

[my_TGM_folder2folder.py](./global_pipeline/my_TGM_folder2folder.py)
- [Input Folder](./global_pipeline/my_TGM_folder2folder.py:296)
- [Output Folder](./global_pipeline/my_TGM_folder2folder.py:297)

In [ ]:
!py ./global_pipeline/my_TGM_folder2folder.py --resume="./maps/models/TGM_nocontrast/best_model.pth" --dataset_name=none --datasets_folder ./maps --G_net unet --GAN_upsample bilinear --GAN_resize 1024 1024

# SHN (Matching)

In [ ]:
%pip install kornia scikit-image wandb openpyxl

[my_myevaluate.py](./local_pipeline/my_myevaluate.py)
- [Input Image](./local_pipeline/my_myevaluate.py:118)
- [Output Image](./local_pipeline/my_myevaluate.py:119)
- [Output Excel](./local_pipeline/my_myevaluate.py:168)

## One-Stage

In [2]:
import numpy

In [5]:
!py -3.13 ./local_pipeline/my_myevaluate.py --dataset_name none --eval_model js_models/1536_one_stage/STHN.pth --val_positive_dist_threshold 512 --lev0 --database_size 1536 --corr_level 4 --test

torch.Size([1, 256, 64, 64]) torch.Size([1, 256, 64, 64])
(840.5726928710938, 649.2992553710938)
tensor([[[ 630.0990,  430.8592],
         [1049.2515,  438.5126],
         [ 624.6422,  867.7606],
         [1058.2981,  860.0647]]])
✅ Done for image 0

📊 Average processing time per image: 0.7620 sec
📁 Saved all corner points to four_point_1_mul6.xlsx


d:\Robotic Perceptoin Lab\map-matching\STHN-JetsonONX8\local_pipeline\model\network.py:110: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast(enabled=self.args.mixed_precision):
C:\Users\Reza\AppData\Local\Python\pythoncore-3.13-64\Lib\site-packages\torch\functional.py:505: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\aten\src\ATen\native\TensorShape.cpp:4319.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
d:\Robotic Perceptoin Lab\map-matching\STHN-JetsonONX8\local_pipeline\model\network.py:139: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast(enabled=self.args.mixed_precision):


## Two-Stages

In [ ]:
!py ./local_pipeline/my_myevaluate.py --dataset_name none --eval_model maps/models/1536_two_stages/STHN.pth --val_positive_dist_threshold 512 --lev0 --database_size 1536 --corr_level 4 --test